<a href="https://colab.research.google.com/github/maya0901/tic-tac-toe/blob/main/charge2ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp /content/drive/MyDrive/Charge2ai/dataset.csv /content/dataset.csv

In [ ]:
!pip install catboost

#START HERE

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
df=pd.read_csv("/content/dataset.csv",)
df

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,...,2,2,2,2,4,2,4,0,20.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,78463,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
25972,71167,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied
25973,37675,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,1,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
25974,90086,Male,Loyal Customer,14,Business travel,Business,1127,3,3,3,...,4,3,2,5,4,5,4,0,0.0,satisfied


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 25976 non-null  int64  
 1   Gender                             25976 non-null  object 
 2   Customer Type                      25976 non-null  object 
 3   Age                                25976 non-null  int64  
 4   Type of Travel                     25976 non-null  object 
 5   Class                              25976 non-null  object 
 6   Flight Distance                    25976 non-null  int64  
 7   Inflight wifi service              25976 non-null  int64  
 8   Departure/Arrival time convenient  25976 non-null  int64  
 9   Ease of Online booking             25976 non-null  int64  
 10  Gate location                      25976 non-null  int64  
 11  Food and drink                     25976 non-null  int

##Scaling Data

In [ ]:
df = df.drop_duplicates()

In [ ]:
from sklearn.preprocessing import RobustScaler

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

X_robust_scaled = df.copy()

scaler = RobustScaler()
X_robust_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df
X_robust_scaled

df['satisfaction'].value_counts()

,count
satisfaction,
neutral or dissatisfied,14573
satisfied,11403


##One Hot Encoding Categorical Values

In [ ]:
df=pd.get_dummies(df)
print(df.columns)

Index(['id', 'Age', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'Gender_Female', 'Gender_Male', 'Customer Type_Loyal Customer',
       'Customer Type_disloyal Customer', 'Type of Travel_Business travel',
       'Type of Travel_Personal Travel', 'Class_Business', 'Class_Eco',
       'Class_Eco Plus', 'satisfaction_neutral or dissatisfied',
       'satisfaction_satisfied'],
      dtype='object')


#Finding Outliers Using z Scores

In [ ]:
#havent tested with new dataset yet

from scipy.stats import zscore
import numpy as np

# zscore=(val-mean)/std, or if using standardscaler, val/mean

numerical_cols = df.select_dtypes(include=np.number).columns
z_scores = df[numerical_cols].apply(lambda x: np.abs(zscore(x, nan_policy='omit')))

threshold = 2.5 # 3.5 seems to be the sweet spot, tho normally it is 2.5.
outlier_rows = df[(z_scores > threshold).any(axis=1)]

print("Rows identified as outliers based on Z-score:")
display(outlier_rows)

Rows identified as outliers based on Z-score:


,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,...,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_neutral or dissatisfied,satisfaction_satisfied
6,79433,77,3987,5,5,5,5,3,5,5,...,False,True,False,True,False,True,False,False,False,True
13,42141,60,451,1,1,4,1,5,5,4,...,False,True,False,True,False,True,False,False,False,True
19,64287,50,3799,5,5,5,5,4,5,4,...,False,True,False,True,False,True,False,False,False,True
42,16172,22,277,3,3,3,3,3,3,3,...,True,True,False,True,False,True,False,False,True,False
87,102623,50,3696,1,1,5,1,2,4,4,...,False,True,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25926,115156,80,337,1,1,4,1,5,5,4,...,False,True,False,True,False,True,False,False,False,True
25932,127293,46,3701,4,4,4,4,4,4,4,...,True,True,False,True,False,True,False,False,False,True
25939,45778,42,3855,1,1,1,1,1,3,1,...,False,True,False,True,False,True,False,False,False,True
25942,91254,44,404,3,1,3,2,3,5,2,...,True,True,False,False,True,False,True,False,True,False


#Handling NaN values

In [ ]:
df = df.fillna(df.mean())

## splitting Dataset and Oversampling

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

X = df.drop(columns=['satisfaction_neutral or dissatisfied',
       'satisfaction_satisfied'])
y = df[['satisfaction_neutral or dissatisfied',
       'satisfaction_satisfied']]

X = df.iloc[:, :-3]
y_onehot = df.iloc[:, -3:]

y = y_onehot.idxmax(axis=1).astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

ros = RandomOverSampler(random_state=42) #smote gives better results

X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

## Making models

In [ ]:
model = GradientBoostingClassifier(loss='log_loss', random_state=42)
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       383
           1       0.95      0.94      0.95      2627
           2       0.93      0.94      0.93      2186

    accuracy                           0.94      5196
   macro avg       0.96      0.96      0.96      5196
weighted avg       0.94      0.94      0.94      5196



In [ ]:
model = GradientBoostingClassifier(loss='log_loss', random_state=42)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       383
           1       0.95      0.94      0.94      2627
           2       0.93      0.94      0.93      2186

    accuracy                           0.94      5196
   macro avg       0.96      0.96      0.96      5196
weighted avg       0.94      0.94      0.94      5196



In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import re

# Sanitize column names
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'\[|\]|<', '_', col)
        new_cols.append(new_col)
    df.columns = new_cols
    return df

X_resampled = sanitize_col_names(X_resampled.copy())
X_test = sanitize_col_names(X_test.copy())

# Initialize and train the XGBoost model
# You might need to tune the hyperparameters for better performance
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_resampled, y_resampled)

X_test_aligned = X_test.reindex(columns=X_resampled.columns, fill_value=0)

y_pred_xgb = xgb_model.predict(X_test_aligned)

# Print classification report
print("Classification Report for XGBoost Model:")
print(classification_report(y_test, y_pred_xgb))

Classification Report for XGBoost Model:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       383
           1       0.95      0.96      0.96      2627
           2       0.96      0.94      0.95      2186

    accuracy                           0.96      5196
   macro avg       0.97      0.97      0.97      5196
weighted avg       0.96      0.96      0.96      5196



In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Initialize and train the CatBoost model
# CatBoost can handle categorical features directly, but our data is already one-hot encoded.
# You might need to tune hyperparameters for better performance
catboost_model = CatBoostClassifier(random_state=42, verbose=0) # verbose=0 to suppress training output
catboost_model.fit(X_resampled, y_resampled)

# Predict on the test set
y_pred_catboost = catboost_model.predict(X_test_aligned)

# Print classification report
print("Classification Report for CatBoost Model:")
print(classification_report(y_test, y_pred_catboost))

Classification Report for CatBoost Model:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       383
           1       0.96      0.96      0.96      2627
           2       0.95      0.95      0.95      2186

    accuracy                           0.96      5196
   macro avg       0.97      0.97      0.97      5196
weighted avg       0.96      0.96      0.96      5196



In [ ]:
catboost_model = CatBoostClassifier(random_state=42, verbose=0) # verbose=0 to suppress training output
catboost_model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_catboost = catboost_model.predict(X_test_aligned)

# Print classification report
print("Classification Report for CatBoost Model:")
print(classification_report(y_test, y_pred_catboost))

Classification Report for CatBoost Model:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       383
           1       0.96      0.96      0.96      2627
           2       0.95      0.95      0.95      2186

    accuracy                           0.96      5196
   macro avg       0.97      0.97      0.97      5196
weighted avg       0.96      0.96      0.96      5196

